# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
pip install pycountry_convert

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install optuna

In [1]:
import pandas as pd
import numpy as np
import re
import pycountry_convert as pc

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

import optuna

import imblearn
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

from sentence_transformers import SentenceTransformer, util  #clustering
#import os
#import csv

import time


/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터 셋 읽어오기

In [26]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [27]:
df_train.head(2)

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True


### EDA

In [28]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

In [19]:
len(df_train[df_train.is_converted==True]) / len(df_train)   #is converted가 True인게 전체 8% 밖에 없다..

0.08178890031872375

In [42]:
df_test.lead_desc_length.describe()

count    5271.000000
mean       81.816164
std       135.063265
min         1.000000
25%         3.000000
50%        26.000000
75%       104.000000
max      1143.000000
Name: lead_desc_length, dtype: float64

In [6]:
# Clustering

# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get all unique sentences from the file
time_sentences = list(set(pd.concat([df_train['expected_timeline'], df_test['expected_timeline']]).unique()))
print(len(time_sentences))
print("Encode the corpus. This might take a while")
time_embeddings = model.encode(time_sentences, batch_size=8, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

# Two parameters to tune:
# min_cluster_size: Only consider cluster that have at least 25 elements
# threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters_time = util.community_detection(time_embeddings, min_community_size=15, threshold=0.4)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

# Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters_time):
    print("\nCluster {}, #{} Elements ".format(i + 1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", time_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", time_sentences[sentence_id])

450
Encode the corpus. This might take a while


Batches: 100%|██████████| 57/57 [00:02<00:00, 22.74it/s]

Start clustering
Clustering done after 0.03 sec

Cluster 1, #113 Elements 
	 discussed with the client. asked to call back on monday will keep a follow up.
	 repeated inquiry from client. he was just inquiring but have not confimred on purchase. we have tried mulitple times but client is not interested in buying.
	 update--14th sept--customer will discuss in detail tomorrow
	 ...
	 not answering call, details shared with rd
	 customer will come for demo in next week, than he will decide further
	 customer has been not  answering call

Cluster 2, #42 Elements 
	 quote send require demo in next week.
	 need to arrange quote and demo.
	 quote send require demo.
	 ...
	 mr rajnikant is allign with the client.
	 requires detail for tender. no purchase requirement right now.
	 customer has not answering call / quotation has been send to customer

Cluster 3, #39 Elements 
	 already in touch with partner.
	 already connect with partner
	 already our customer, have shared prices for new require

In [7]:
for i, cluster in enumerate(clusters_time):
    for sentence_id in cluster:
        if "didn't respond to calls" == time_sentences[sentence_id]:
            print("cluster:", i)
            
#새로운 데이터에 대해서는 유사도 matrix 활용

cluster: 0


In [8]:
# Clustering

# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get all unique sentences from the file
inquiry_sentences = list(set(pd.concat([df_train['inquiry_type'], df_test['inquiry_type']]).unique()))
print("Encode the corpus. This might take a while")
inquiry_embeddings = model.encode(inquiry_sentences, batch_size=8, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

# Two parameters to tune:
# min_cluster_size: Only consider cluster that have at least 25 elements
# threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters_inquiry = util.community_detection(inquiry_embeddings, min_community_size=4, threshold=0.5)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

# Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters_inquiry):
    print("\nCluster {}, #{} Elements ".format(i + 1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", inquiry_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", inquiry_sentences[sentence_id])

Encode the corpus. This might take a while


Batches: 100%|██████████| 9/9 [00:00<00:00, 44.49it/s]

Start clustering
Clustering done after 0.00 sec

Cluster 1, #13 Elements 
	 Usage or technical consultation
	 usage or technical consultation
	 Usage or Technical Consultation
	 ...
	 Quotation or Purchase Consultation
	 quotation_or_purchase_consultation
	 technical

Cluster 2, #5 Elements 
	 other_
	 other
	 Other
	 ...
	 Other
	 Others
	 others

Cluster 3, #4 Elements 
	 Request for quotation or purchase
	 Purchase or Quotation
	 quotation_
	 ...
	 Purchase or Quotation
	 quotation_
	 Request for Distributorship

Cluster 4, #4 Elements 
	 Sales Inquiry
	 Sales inquiry
	 sales
	 ...
	 Sales inquiry
	 sales
	 Purchase


In [9]:
# Clustering

# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get all unique sentences from the file
job_sentences = list(set(pd.concat([df_train['customer_job'], df_test['customer_job']]).unique()))
print(len(job_sentences))
print("Encode the corpus. This might take a while")
job_embeddings = model.encode(job_sentences, batch_size=8, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

# Two parameters to tune:
# min_cluster_size: Only consider cluster that have at least 25 elements
# threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters_job = util.community_detection(job_embeddings, min_community_size=5, threshold=0.5)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

# Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters_job):
    print("\nCluster {}, #{} Elements ".format(i + 1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", job_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", job_sentences[sentence_id])

563
Encode the corpus. This might take a while


Batches: 100%|██████████| 71/71 [00:01<00:00, 50.80it/s]


Start clustering
Clustering done after 0.02 sec

Cluster 1, #70 Elements 
	 project designer
	 project manager/designer
	 designer/ project manager
	 ...
	 project manager / estimator
	 designer/installer
	 creative director

Cluster 2, #23 Elements 
	 building owner
	 contractor/owner
	 managing director
	 ...
	 purchasing manager
	 site manager
	 director purchaser

Cluster 3, #20 Elements 
	 coordinator
	 service coordinator
	 marketing coordinator
	 ...
	 projection manager
	 office manager
	 manager

Cluster 4, #19 Elements 
	 it director
	 director of finance
	 director,it
	 ...
	 ceo/founder
	 executive
	 director of lodging

Cluster 5, #17 Elements 
	 head of technology
	 tech
	 information_technology
	 ...
	 design and installation company
	 equipment planner
	 technical sales

Cluster 6, #13 Elements 
	 engineering
	 principal engineer
	 engineering & technical executive
	 ...
	 technical / decision maker
	 product researcher
	 general contractor

Cluster 7, #13 Elements 
	 o

In [10]:
# Clustering

# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get all unique sentences from the file
product_sentences = list(set(pd.concat([df_train['product_category'], df_test['product_category']]).unique()))
print(len(product_sentences))
print("Encode the corpus. This might take a while")
product_embeddings = model.encode(product_sentences, batch_size=8, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

# Two parameters to tune:
# min_cluster_size: Only consider cluster that have at least 25 elements
# threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters_product = util.community_detection(product_embeddings, min_community_size=10, threshold=0.5)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

# Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters_product):
    print("\nCluster {}, #{} Elements ".format(i + 1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", product_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", product_sentences[sentence_id])

360
Encode the corpus. This might take a while


Batches: 100%|██████████| 45/45 [00:01<00:00, 43.13it/s]


Start clustering
Clustering done after 0.01 sec

Cluster 1, #76 Elements 
	 monitor signage,commercial tv,monior/monitor tv,projector,tv
	 monitor signage,commercial tv,monior/monitor tv,pc,projector,tv,audio/video
	 monitor signage,commercial tv,monior/monitor tv,tv
	 ...
	 system ac,tv
	 collaboration displays
	 projector,ems

Cluster 2, #26 Elements 
	 system ac,aircare
	 system ac,chiller
	 chiller,aircare
	 ...
	 chiller,dryer
	 on or several reversible ac
	 dryer,chiller

Cluster 3, #22 Elements 
	 vrf,single-split,chiller
	 vrf,multi-split,chiller
	 vrf,multi-split,single-split,chiller
	 ...
	 multi split
	 multi-split,single-split
	 multi-split

Cluster 4, #20 Elements 
	 55us660h0sd.bwz
	 50us660h0sd.bwz
	 43us660h0sd.awz
	 ...
	 id
	 gscd046
	 ur640

Cluster 5, #12 Elements 
	 pc,robot,system ac,chiller,tv,refrigerator,washing machine,vacuum cleaner,styler,dryer,mobile,audio/
	 tv,refrigerator,washing machine,vacuum cleaner,audio/video
	 projector,system ac,water care
	 ...
	

In [11]:
# Clustering

# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get all unique sentences from the file
position_sentences = list(set(pd.concat([df_train['customer_position'], df_test['customer_position']]).unique()))
print(len(position_sentences))
print("Encode the corpus. This might take a while")
position_embeddings = model.encode(position_sentences, batch_size=8, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

# Two parameters to tune:
# min_cluster_size: Only consider cluster that have at least 25 elements
# threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters_position = util.community_detection(position_embeddings, min_community_size=5, threshold=0.5)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

# Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters_position):
    print("\nCluster {}, #{} Elements ".format(i + 1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", position_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", position_sentences[sentence_id])

118
Encode the corpus. This might take a while


Batches: 100%|██████████| 15/15 [00:00<00:00, 48.97it/s]

Start clustering
Clustering done after 0.00 sec

Cluster 1, #23 Elements 
	 academic specialist
	 education professional
	 surgery professional
	 ...
	 academic coordinator/ post graduate teacher (accountancy, business studies)/ tgt (ict)
	 intern
	 teacher/middle school coordinator

Cluster 2, #8 Elements 
	 maths lecturer
	 associate professor in electronics engg
	 guest faculty
	 ...
	 math and physics teacher
	 assistant professor of enlish
	 physics and mathematics teacher

Cluster 3, #8 Elements 
	 administrative
	 chief executive officer
	 manager
	 ...
	 government
	 employee
	 chairman

Cluster 4, #8 Elements 
	 business partner
	 partner
	 consultant
	 ...
	 commercial consultant
	 business development
	 business development/sales

Cluster 5, #5 Elements 
	 ceo/founder
	 ceo/fundador
	 founder
	 ...
	 founder
	 director
	 co-founder


In [60]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)
def extract_last_segment(text):
    if not isinstance(text, str):  # 문자열 아닌 경우 제거
        return None
    match = re.search(r'[^/]+$', text)
    if '@' in text:
        return None
    elif match:
        return match.group()
    else:
        return None

def change_country(country_name):
    correction_dict = {
    'U.A.E': 'United Arab Emirates',
    ' Connecticut United States': 'United States',
    'Czech': 'Czechia',
    'Barranquilla': 'Colombia',
    'ARGENTINA': 'Argentina',
    'EGYPT': 'Egypt',
    ' mumbai': 'India',
    'Hacienda Heights, California, United States': 'United States',
    'US': 'United States'
}
    # country_name이 correction_dict의 키에 해당하면 해당 값으로 교체
    country_name = correction_dict.get(country_name, country_name)
    return country_name

def country_to_continent(country_name):
    country_name = change_country(country_name)
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return ""  #test에서 결측치 나타나서 ""으로 채움.

def timeline(str):
    for i, cluster in enumerate(clusters_time):
        for sentence_id in cluster:
            if str == time_sentences[sentence_id]:
                return i

def inquiry(str):
    for i, cluster in enumerate(clusters_inquiry):
        for sentence_id in cluster:
            if str == inquiry_sentences[sentence_id]:
                return i
            
def job(str):
    for i, cluster in enumerate(clusters_job):
        for sentence_id in cluster:
            if str == job_sentences[sentence_id]:
                return i

def product(str):
    for i, cluster in enumerate(clusters_product):
        for sentence_id in cluster:
            if str == product_sentences[sentence_id]:
                return i

def position(str):
    for i, cluster in enumerate(clusters_position):
        for sentence_id in cluster:
            if str == position_sentences[sentence_id]:
                return i
            
def preprocessing(df):
    #나라 비교
    #df_train[df_train['customer_country'] != df_train['customer_country.1']][['customer_country', 'customer_country.1']].isnull().sum()
    ###    => .isnull().sum() 없는 것과 비교하면, 두 칼럼이 다른 경우는 오직 null 일 때!!
    ###    => 칼럼 하나만 사용!
    df['country'] = df['customer_country'].apply(extract_last_segment)
    df.country = df.country.fillna("")
    df.drop(['customer_country','customer_country.1', 'response_corporate'], axis=1, inplace=True)
    
    df['continent'] = df.country.apply(country_to_continent)
 
    #com_reg_ver_win_rate가 같은 국가끼리 같다.
    
    
    #customer_type 결측치 많고 채울 방법이 없어보임
    df.drop(['customer_type'], axis=1, inplace=True)
    
    df.drop(['historical_existing_cnt'], axis=1, inplace=True)
    
    #id_strategic_ver (business unit == ID) , it_strategic_ver (business unit==IT)
    # 가중치 부여 여부. 0이 하나도 없음. 결측치는 부여를 안 했으니 0으로 채우기.
    df['id_strategic_ver'] = df['id_strategic_ver'].fillna(0)
    df['it_strategic_ver'] = df['it_strategic_ver'].fillna(0)
    #idit는 id와 it 둘 중 하나가 1이면 1인 것으로, feature importance 확인 후 중요하지 않으면 제거
    df.drop('idit_strategic_ver', inplace=True, axis=1) #다중공산성
    #df['idit_strategic_ver'] = df['idit_strategic_ver'].fillna(0)
        
    df.drop(['product_subcategory', 'product_modelname'], axis=1, inplace=True)
    
    #expected_timeline  / inquiry_type
    df['timeline'] = df['expected_timeline'].apply(timeline)
    df['inquiry'] = df['inquiry_type'].apply(inquiry)
    df['customer_job'].fillna("other", inplace=True)
    df['customer_job'] = df['customer_job'].apply(job)
    df['product_category'].fillna("", inplace=True) 
    df['product'] = df['product_category'].apply(product)
    df['customer_position'] = df['customer_position'].apply(position)
    #미분류 -1
    col_cluster = ['timeline', 'inquiry', 'customer_job', 'product', 'customer_position']
    df[col_cluster] = df[col_cluster].fillna(-1)
    col_cluster = ['timeline', 'inquiry', 'customer_job', 'product', 'customer_position']
    df.drop(['expected_timeline', 'inquiry_type', 'product_category'], axis=1, inplace=True)
    #df[col_cluster] = df[col_cluster].astype(int)
    
    df.drop(['business_area', 'business_subarea'], axis=1, inplace=True)
    
    ####   log변환으로 분포 바꿔보기
    df['lead_desc_length'] = np.log1p(df['lead_desc_length'])
    
    
    #index의 경우 숫자가 커서..각각이 전체에서 몇 번 나타난지를 따짐.
    customer_counts = df['customer_idx'].value_counts()
    lead_counts = df['lead_owner'].value_counts()
    df['customer_count'] = df['customer_idx'].map(customer_counts)
    df['lead_count'] = df['lead_owner'].map(lead_counts)
    df.drop(['customer_idx', 'lead_owner'], axis=1, inplace=True)
    
    
    
    #feature selection
    drop_feature = ['com_reg_ver_win_rate', 'ver_win_rate_x', 'ver_win_ratio_per_bu']
    df.drop(drop_feature, axis=1, inplace=True)
    
    
    return df

df_train = preprocessing(df_train)
df_train = df_train.dropna(subset='continent')

df_test = preprocessing(df_test)



In [61]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5271 entries, 0 to 5270
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5271 non-null   int64  
 1   bant_submit        5271 non-null   float64
 2   business_unit      5271 non-null   object 
 3   enterprise         5271 non-null   object 
 4   id_strategic_ver   5271 non-null   float64
 5   it_strategic_ver   5271 non-null   float64
 6   customer_job       5271 non-null   float64
 7   lead_desc_length   5271 non-null   float64
 8   customer_position  5271 non-null   float64
 9   ver_cus            5271 non-null   int64  
 10  ver_pro            5271 non-null   int64  
 11  is_converted       5271 non-null   int64  
 12  country            5271 non-null   object 
 13  continent          5271 non-null   object 
 14  timeline           5271 non-null   float64
 15  inquiry            5271 non-null   float64
 16  product            5271 

In [14]:
df_train[df_train['business_unit']=='ID']

,bant_submit,business_unit,com_reg_ver_win_rate,enterprise,id_strategic_ver,it_strategic_ver,customer_job,lead_desc_length,customer_position,ver_cus,...,ver_win_rate_x,ver_win_ratio_per_bu,is_converted,country,continent,timeline,inquiry,product,customer_count,lead_count
388,1.00,ID,0.057534,Enterprise,1.0,0.0,2.0,"[0, 250)",4.0,1,...,0.003079,0.064566,True,India,Asia,4.0,0.0,0.0,1,61
389,1.00,ID,0.057534,Enterprise,1.0,0.0,8.0,"[0, 250)",4.0,1,...,0.003079,0.064566,True,India,Asia,4.0,0.0,0.0,1,37
390,1.00,ID,0.057534,Enterprise,1.0,0.0,11.0,"[0, 250)",4.0,1,...,0.003079,0.064566,True,India,Asia,4.0,0.0,0.0,1,51
391,1.00,ID,0.057534,SMB,1.0,0.0,3.0,"[0, 250)",2.0,1,...,0.003079,0.064566,True,India,Asia,4.0,0.0,0.0,1,216
392,1.00,ID,0.057534,Enterprise,1.0,0.0,-1.0,"[0, 250)",2.0,1,...,0.003079,0.064566,True,India,Asia,4.0,0.0,0.0,1,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59157,0.25,ID,NaN,SMB,0.0,0.0,-1.0,"[0, 250)",NaN,0,...,0.000026,0.031579,False,Germany,Europe,-1.0,0.0,NaN,2,305
59158,0.25,ID,NaN,SMB,0.0,0.0,-1.0,"[0, 250)",NaN,0,...,0.000026,0.031579,False,Germany,Europe,-1.0,0.0,NaN,2,305
59159,0.25,ID,NaN,Enterprise,0.0,0.0,8.0,"[0, 250)",NaN,0,...,0.000026,0.031579,False,Germany,Europe,-1.0,0.0,0.0,2,305
59160,0.25,ID,NaN,Enterprise,0.0,0.0,8.0,"[0, 250)",NaN,0,...,0.000026,0.031579,False,Germany,Europe,-1.0,0.0,0.0,2,305


In [15]:
# EDA   :    com_reg_ver_win_rate에 따라 국가 분류
country_by_win_rate = {}
values = df_train['com_reg_ver_win_rate'].unique()
for i in values:
    country_by_win_rate[i] = list(df_train[df_train['com_reg_ver_win_rate'] == i]['country'].unique())

print(country_by_win_rate) # country 열이 uncleaned 된 상태.. -> unique()

{0.0666666666666666: ['Philippines', 'Singapore', 'Australia', '', 'Indonesia', 'Bangladesh', 'Papua New Guinea', 'Sri Lanka', 'India', 'Thailand', 'Vietnam', 'China'], 0.0888888888888888: ['India'], 0.0408163265306122: ['Nigeria', 'Saudi Arabia', 'U.A.E', 'South Africa', 'Ghana', 'Egypt', 'Democratic Republic of the Congo', 'Ethiopia', 'Kenya', 'Oman', 'Pakistan', '', 'United Republic of Tanzania', 'Qatar', 'Afghanistan', 'Mozambique', 'Turkey', 'Türkiye', 'Togo', 'Jordan', 'Iraq', 'Israel', 'Mauritania'], 0.0039370078740157: ['Brazil', 'Colombia', 'Mexico', 'Guatemala', 'Panama', 'Chile', 'El Salvador', 'Uruguay', 'Peru', 'Jamaica', 'Belize', 'Argentina', '', 'Antigua', 'Dominican Republic', 'Honduras', 'Bahamas', 'Colombia C2: Solo información'], nan: [], 0.0575342465753424: ['India', ' mumbai', ''], 0.075: ['South Africa', 'Saudi Arabia', 'U.A.E', 'Nigeria', 'Senegal', 'Afghanistan', 'Kuwait', 'Egypt', 'Turkey', 'Kenya', 'Ethiopia', 'Oman', 'Pakistan', 'Algeria', 'Israel', 'Qatar',

In [16]:
df_train['com_reg_ver_win_rate'] #Vertical level1(business_area), business_unit, region(country)에 의해 계산됨
#기준이 더 있는 거 같아서 일단 보류

0        0.066667
1        0.066667
2        0.088889
3        0.088889
4        0.088889
           ...   
59294         NaN
59295    0.040000
59296    0.040000
59297    0.040000
59298         NaN
Name: com_reg_ver_win_rate, Length: 59299, dtype: float64

In [ ]:
df_train.sort_values(by=['com_reg_ver_win_rate','business_area','business_unit','country'])[['business_area','city', 'business_unit', 'com_reg_ver_win_rate']][1513:1522]

In [ ]:
df_train[(df_train['business_area']=='retail') & (df_train['city']=='Quezon City') & (df_train['business_unit']=='AS')]

In [ ]:
#결측치 채우기
'''
for i in df_train[df_train['com_reg_ver_win_rate'].isnull()].index:
    ba = df_train['business_area'][i]
    city = df_train['city'][i]
    bu = df_train['business_unit'][i]
    
    #보류!
    val = df_train[(df_train['business_area']==ba) & (df_train['city']==city) & (df_train['business_unit']==bu)][0]
    df_train['com_reg_ver_win_rate'][i] = val
    '''

In [ ]:
'''
#customer idx
df_train['customer_idx'].value_counts #32160개
df_train.groupby('customer_idx').count()  #bant_submit 결측치 없음. -> 기준으로 이용
df_train['customer_cnt'] = 0 #초기화

pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning
for i in df_train.index:
    df_train['customer_cnt'][i] = df_train.groupby('customer_idx').count()['bant_submit'][df_train['customer_idx'][i]]
df_train.drop('customer_idx', axis=1, inplace=True)
'''

In [ ]:
#특정 갯수 이상 열 제거
#df_train = df_train.dropna(axis=1, thresh=40000)

In [ ]:
df_train.info()

In [ ]:
# => customer job / inquire_type / costomer_position / response_corporate 갯수 많다 + multi lang 존재.
print('customer_country', df_train['customer_country'].unique())
print('business_unit', df_train['business_unit'].unique())
print('enterprise', df_train['enterprise'].unique())
print('customer_job', df_train['customer_job'].unique())
print('inquiry_type', df_train['inquiry_type'].unique())
print('customer_country.1', df_train['customer_country.1'].unique())
print('customer_position', df_train['customer_position'].unique())
print('response_corporate', df_train['response_corporate'].unique())

## 2. 데이터 전처리

### 레이블 인코딩

In [62]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bant_submit        59299 non-null  float64
 1   business_unit      59299 non-null  object 
 2   enterprise         59299 non-null  object 
 3   id_strategic_ver   59299 non-null  float64
 4   it_strategic_ver   59299 non-null  float64
 5   customer_job       59299 non-null  float64
 6   lead_desc_length   59299 non-null  float64
 7   customer_position  59299 non-null  float64
 8   ver_cus            59299 non-null  int64  
 9   ver_pro            59299 non-null  int64  
 10  is_converted       59299 non-null  bool   
 11  country            59299 non-null  object 
 12  continent          59299 non-null  object 
 13  timeline           59299 non-null  float64
 14  inquiry            59299 non-null  float64
 15  product            59299 non-null  float64
 16  customer_count     592

In [63]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [64]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "business_unit",
    "enterprise",
    #"lead_desc_length",
#    "business_subarea",
#    "business_area",  
    #"product_category",
    #"customer_position",
    "country",
    "continent"
    
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [65]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [66]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

### 2-2. 학습, 검증 데이터 분리

In [67]:
len(df_train['is_converted'])

59299

In [68]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    #random_state=400,
    stratify=df_train['is_converted']
)

In [69]:
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE, SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks, OneSidedSelection
# define dataset
# summarize class distribution
counter = Counter(y_train)
print(counter)


# transform the dataset
#oversample = BorderlineSMOTE()
#oversample = SMOTEENN()
#oversample = SMOTE()
#x_train, y_train = oversample.fit_resample(x_train, y_train)

undersample = OneSidedSelection()
x_train, y_train = undersample.fit_resample(x_train, y_train)

# summarize the new class distribution
counter = Counter(y_train)
print(counter)

oversample = BorderlineSMOTE()
x_train, y_train = oversample.fit_resample(x_train, y_train)

# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({False: 43559, True: 3880})
Counter({False: 43054, True: 3880})
Counter({False: 43054, True: 43054})


## 3. 모델 학습

### 모델 성능 보기

In [119]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [126]:
#Balancd Random Forest Classifier (Ensemble)
from imblearn.ensemble import BalancedRandomForestClassifier

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 10, 100),
        #"min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        #"max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 5, 20),
        "bootstrap": False,
        "warm_start": True,
        #"class_weight": "balanced",
        "random_state" : 44,
        "replacement" : True,
        "sampling_strategy" : "all"
    }

    brfc = BalancedRandomForestClassifier(**params)
    brfc.fit(x_train, y_train)
    preds = brfc.predict(x_val)
    pred_labels = np.where(preds >= 0.5, 1, 0)
    recall = recall_score(y_val, pred_labels)
    return recall

brfc_study = optuna.create_study(direction='maximize')


[I 2024-02-16 08:58:03,218] A new study created in memory with name: no-name-0ff0ccdd-6858-4a74-9e4e-b40b384aa054


In [127]:
brfc_study.optimize(objective, n_trials=50)

[I 2024-02-16 08:58:25,547] Trial 0 finished with value: 0.7731958762886598 and parameters: {'n_estimators': 198, 'max_depth': 87}. Best is trial 0 with value: 0.7731958762886598.
[I 2024-02-16 08:59:35,406] Trial 1 finished with value: 0.7752577319587629 and parameters: {'n_estimators': 668, 'max_depth': 46}. Best is trial 1 with value: 0.7752577319587629.
[I 2024-02-16 09:00:07,580] Trial 2 finished with value: 0.7742268041237114 and parameters: {'n_estimators': 310, 'max_depth': 73}. Best is trial 1 with value: 0.7752577319587629.
[I 2024-02-16 09:00:20,855] Trial 3 finished with value: 0.7752577319587629 and parameters: {'n_estimators': 128, 'max_depth': 85}. Best is trial 1 with value: 0.7752577319587629.
[I 2024-02-16 09:01:46,429] Trial 4 finished with value: 0.8072164948453608 and parameters: {'n_estimators': 847, 'max_depth': 21}. Best is trial 4 with value: 0.8072164948453608.
[I 2024-02-16 09:03:30,163] Trial 5 finished with value: 0.7762886597938145 and parameters: {'n_esti

In [128]:
brfc = BalancedRandomForestClassifier(**brfc_study.best_trial.params)
brfc.fit(x_train, y_train)
pred_brfc = brfc.predict(x_val)
get_clf_eval(y_val, pred_brfc)

/home/elicer/.local/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:576: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/elicer/.local/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:588: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/elicer/.local/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:600: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  wa

오차행렬:
 [[ 878   92]
 [1593 9297]]

정확도: 0.8579
정밀도: 0.3553
재현율: 0.9052
F1: 0.5103


In [129]:
#DecisionTree

def objective(trial):
    class_weight_options = [
        {0: 1, 1: 7},
        {0: 1, 1: 5},
        {0: 1, 1: 3},
        {0: 1, 1: 1}
    ]
    
    params = {
        "max_depth": trial.suggest_int("max_depth", 7, 25),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 6),
        "class_weight": trial.suggest_categorical("class_weight", class_weight_options),
        #"max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 5, 20),
        "random_state" : 44
    }

    dtc = DecisionTreeClassifier(**params)
    dtc.fit(x_train, y_train)
    preds = dtc.predict(x_val)
    pred_labels = np.where(preds >= 0.5, 1, 0)
    recall = recall_score(y_val, pred_labels)
    return recall

dtc_study = optuna.create_study(direction='maximize')


[I 2024-02-16 09:44:58,505] A new study created in memory with name: no-name-6efade76-ff66-4ec9-89b1-3f79fd200389


In [130]:
dtc_study.optimize(objective, n_trials=50)

/home/elicer/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 7} which is of type dict.
  warnings.warn(message)
/home/elicer/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 5} which is of type dict.
  warnings.warn(message)
/home/elicer/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 3} which is of type dict.
  warnings.warn(message)
/home/elicer/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, 

In [131]:
dtc = DecisionTreeClassifier(**dtc_study.best_trial.params)
dtc.fit(x_train, y_train)
pred_dtc = dtc.predict(x_val)
get_clf_eval(y_val, pred_dtc)

오차행렬:
 [[ 953   17]
 [3937 6953]]

정확도: 0.6666
정밀도: 0.1949
재현율: 0.9825
F1: 0.3253


In [132]:
###############################################################
#XGBoost
#https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-categorical-feature

def objective(trial):
    params = {
        "device" : "cuda",
        "objective": "binary:logistic",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-2, 0.5),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_child_weight": trial.suggest_int("min_child_weight", 2, 5),
        "subsample": trial.suggest_float("subsample", 0.3, 0.7),
        #"colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "lambda": trial.suggest_float("lambda", 0.7, 2.0)
        
    }

    xgbc = xgb.XGBClassifier(**params)
    xgbc.fit(x_train, y_train, verbose=False)
    preds = xgbc.predict(x_val)
    #pred_labels = np.rint(preds)
    pred_labels = np.where(preds >= 0.5, 1, 0)
    recall = recall_score(y_val, pred_labels)
    return recall

xgbc_study = optuna.create_study(direction='maximize')


[I 2024-02-16 09:45:14,519] A new study created in memory with name: no-name-f53582cf-e1cc-4c1c-87f1-8307cf767d6a


In [133]:
xgbc_study.optimize(objective, n_trials=100)

[I 2024-02-16 09:45:15,254] Trial 0 finished with value: 0.8474226804123711 and parameters: {'n_estimators': 226, 'learning_rate': 0.13495388743161754, 'max_depth': 3, 'min_child_weight': 4, 'subsample': 0.6446267991830453, 'lambda': 1.7924206244044611}. Best is trial 0 with value: 0.8474226804123711.
[I 2024-02-16 09:45:20,398] Trial 1 finished with value: 0.7917525773195876 and parameters: {'n_estimators': 830, 'learning_rate': 0.03229602281106599, 'max_depth': 15, 'min_child_weight': 3, 'subsample': 0.34480248392867513, 'lambda': 0.7695534150441978}. Best is trial 0 with value: 0.8474226804123711.
[I 2024-02-16 09:45:23,433] Trial 2 finished with value: 0.7927835051546391 and parameters: {'n_estimators': 568, 'learning_rate': 0.23904920263287188, 'max_depth': 14, 'min_child_weight': 3, 'subsample': 0.3157868345584553, 'lambda': 1.8236279313005157}. Best is trial 0 with value: 0.8474226804123711.
[I 2024-02-16 09:45:23,861] Trial 3 finished with value: 0.7938144329896907 and paramete

In [134]:
xgbc = xgb.XGBClassifier(**xgbc_study.best_trial.params)
xgbc.fit(x_train, y_train)
pred_xgbc = xgbc.predict(x_val)
get_clf_eval(y_val, pred_xgbc)

오차행렬:
 [[ 941   29]
 [3587 7303]]

정확도: 0.6951
정밀도: 0.2078
재현율: 0.9701
F1: 0.3423


In [135]:
for val, col in zip(xgbc.feature_importances_.flatten(), x_train.columns):
    print('columns:', col, '          feature_importance:', val)


columns: bant_submit           feature_importance: 0.03287434
columns: business_unit           feature_importance: 0.037000056
columns: enterprise           feature_importance: 0.0
columns: id_strategic_ver           feature_importance: 0.008672206
columns: it_strategic_ver           feature_importance: 0.056280218
columns: customer_job           feature_importance: 0.0037673174
columns: lead_desc_length           feature_importance: 0.040830404
columns: customer_position           feature_importance: 0.02855423
columns: ver_cus           feature_importance: 0.0
columns: ver_pro           feature_importance: 0.030028868
columns: country           feature_importance: 0.05535065
columns: continent           feature_importance: 0.06514617
columns: timeline           feature_importance: 0.07996727
columns: inquiry           feature_importance: 0.12735896
columns: product           feature_importance: 0.058574323
columns: customer_count           feature_importance: 0.024166344
columns: lea

In [81]:
#randomforest classifier

def objective(trial):
    param = {
        #'n_estimators': trial.suggest_int('n_estimators', 1000, 1200),
        #'n_estimators': 1000,
        #'criterion': 'entropy',
        'max_depth' : trial.suggest_int('max_depth', 7, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 6),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),
        'max_features': 1.0,
        #'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': True,
        'class_weight': 'balanced',
        #'n_jobs': None,
        'random_state': 44
        }


    rfc = RandomForestClassifier(**param)
    rfc.fit(x_train, y_train)
    preds = rfc.predict(x_val)
    pred_labels = np.rint(preds)
    f1score = f1_score(y_val, pred_labels)
    return f1score


rfc_study = optuna.create_study(direction="maximize")


[I 2024-02-16 07:14:54,791] A new study created in memory with name: no-name-68b2e173-7f6d-4866-bec7-7448c497ba16


In [82]:
rfc_study.optimize(objective, n_trials=50, timeout=700)

print("Number of finished trials: ", len(rfc_study.trials))
print("Best trial:")
trial = rfc_study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-02-16 07:15:13,596] Trial 0 finished with value: 0.5907973305233579 and parameters: {'max_depth': 13, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.5907973305233579.
[I 2024-02-16 07:15:32,399] Trial 1 finished with value: 0.5907973305233579 and parameters: {'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.5907973305233579.
[I 2024-02-16 07:15:49,649] Trial 2 finished with value: 0.5301131152552736 and parameters: {'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.5907973305233579.
[I 2024-02-16 07:16:09,126] Trial 3 finished with value: 0.6149812734082397 and parameters: {'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 3 with value: 0.6149812734082397.
[I 2024-02-16 07:16:22,289] Trial 4 finished with value: 0.3612679114198871 and parameters: {'max_depth': 7, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 3 wit

Number of finished trials:  38
Best trial:
  Value: 0.6521394611727416
  Params: 
    max_depth: 15
    min_samples_split: 3
    min_samples_leaf: 2


In [83]:
rfc = RandomForestClassifier(**rfc_study.best_trial.params)
rfc.fit(x_train, y_train)
pred_rfc = rfc.predict(x_val)
get_clf_eval(y_val, pred_rfc)

오차행렬:
 [[  819   151]
 [  693 10197]]

정확도: 0.9288
정밀도: 0.5417
재현율: 0.8443
F1: 0.6600


## 4. 제출하기

### 테스트 데이터 예측

In [136]:
#최종 학습
xgbc = xgb.XGBClassifier(**xgbc_study.best_trial.params)
xgbc.fit(pd.concat([x_train, x_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.9704687598758174,
              learning_rate=0.010251646163918996, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=123, n_jobs=None,
              num_parallel_tree=None, ...)

In [137]:
brfc = BalancedRandomForestClassifier(**brfc_study.best_trial.params)
brfc.fit(pd.concat([x_train, x_val]), pd.concat([y_train, y_val]))

/home/elicer/.local/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:576: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/elicer/.local/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:588: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/elicer/.local/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:600: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  wa

BalancedRandomForestClassifier(max_depth=10, n_estimators=644)

In [138]:
dtc = DecisionTreeClassifier(**dtc_study.best_trial.params)
dtc.fit(pd.concat([x_train, x_val]), pd.concat([y_train, y_val]))

DecisionTreeClassifier(class_weight={0: 1, 1: 7}, max_depth=7,
                       min_samples_split=4)

In [139]:
x_test = df_test.drop(["is_converted", "id"], axis=1)   # 예측에 필요한 데이터 분리
test_pred1 = xgbc.predict(x_test.fillna(0))   #test_pred
test_pred2 = brfc.predict(x_test.fillna(0))   #test_pred
test_pred3 = dtc.predict(x_test.fillna(0))   #test_pred

In [140]:
test_pred = np.where(((test_pred1+test_pred2+test_pred3)/3 ) >=0.2, 1, 0) #3개 중 1이 1개이상이면 1
sum(test_pred) # True로 예측된 개수

4104

In [141]:
test_pred

array([0, 1, 1, ..., 0, 1, 1])

### 제출 파일 작성

In [142]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**